## Nino SST Indices (Nino 1+2, 3, 3.4, 4; ONI and TNI)
https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni

Niño 1+2 (0-10S, 90W-80W):  The Niño 1+2 region is the smallest and eastern-most of the Niño SST regions, and corresponds with the region of coastal South America where El Niño was first recognized by the local populations.  This index tends to have the largest variance of the Niño SST indices.

Niño 3 (5N-5S, 150W-90W):  This region was once the primary focus for monitoring and predicting El Niño, but researchers later learned that the key region for coupled ocean-atmosphere interactions for ENSO lies further west (Trenberth, 1997).  Hence, the Niño 3.4 and ONI became favored for defining El Niño and La Niña events.

Niño 3.4 (5N-5S, 170W-120W):  The  Niño 3.4 anomalies may be thought of as representing the average equatorial SSTs across the Pacific from about the dateline to the South American coast.  The Niño 3.4 index typically uses a 5-month running mean, and El Niño or La  Niña events are defined when the  Niño 3.4 SSTs exceed +/- 0.4C for a period of six months or more.

ONI (5N-5S, 170W-120W): The ONI uses the same region as the Niño 3.4 index.  The ONI uses a 3-month running mean, and to be classified as a full-fledged El Niño or La Niña, the anomalies must exceed +0.5C or -0.5C for at least five consecutive months.  This is the operational definition used by NOAA.

Niño 4 (5N-5S, 160E-150W): The  Niño 4 index captures SST anomalies in the central equatorial Pacific.  This region tends to have less variance than the other Niño regions.

In [1]:
# load packages
import s3fs; import xarray as xr; import numpy as np; import pandas as pd; import dask.array as da
import matplotlib.pyplot as plt; import cartopy.crs as ccrs
import netCDF4 as nc; import datetime as dt; import scipy
import warnings; import expectexception
warnings.filterwarnings('ignore')
import intake; import pprint
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import HTML

In [2]:
# Open original collection description file
cat_url_orig = '/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm2-le.json'
coll_orig = intake.open_esm_datastore(cat_url_orig)
subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical')
member_id_list = subset.df.member_id.unique()

In [ ]:
%%time

index_nino12_full = []
normalized_index_nino12_full = []

index_nino3_full = []
normalized_index_nino3_full = []

index_nino34_full = []
index_nino34_rolling_mean_full = []
normalized_index_nino34_rolling_mean_full = []

index_nino4_full = []
normalized_index_nino4_full = []

index_oni_full = []
index_oni_rolling_mean_full = []
normalized_index_oni_rolling_mean_full = []

for i in range(50,100):
# for i in range(5):
    subset = coll_orig.search(component='atm',variable='SST',frequency='month_1',experiment='historical',member_id= str(member_id_list[i]))
    dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True}, storage_options={"anon": True})
    # ds = dsets['atm.historical.cam.h0.cmip6.SST'] # before 50
    ds = dsets['atm.historical.cam.h0.smbb.SST'] # after 50 # Ask Liz
    SST = ds.SST.isel(member_id=0).load()
    
    # Niño 1+2 (5N-5S, 270W-90W)
    tos_nino12 = SST.sel(lat=slice(0, 10), lon=slice(270, 280))
    gb = tos_nino12.groupby('time.month')
    tos_nino12_anom = gb - gb.mean(dim='time')
    index_nino12 = tos_nino12_anom.mean(dim=['lat', 'lon'])
    std_dev_tos_12 = tos_nino12.std()
    normalized_index_nino12 = index_nino12 / std_dev_tos_12
    
    index_nino12_full.append(index_nino12)
    normalized_index_nino12_full.append(normalized_index_nino12)
    
    # Niño 3 (5N-5S, 150W-90W)
    tos_nino3 = SST.sel(lat=slice(-5, 5), lon=slice(210, 270))
    gb = tos_nino3.groupby('time.month')
    tos_nino3_anom = gb - gb.mean(dim='time')
    index_nino3 = tos_nino3_anom.mean(dim=['lat', 'lon'])
    std_dev_tos_3 = tos_nino3.std()
    normalized_index_nino3_mean = index_nino3 / std_dev_tos_3
    
    index_nino3_full.append(index_nino3)
    normalized_index_nino3_full.append(normalized_index_nino3_mean)
    
    # Niño 3.4 (5N-5S, 170W-120W)
    tos_nino34 = SST.sel(lat=slice(-5, 5), lon=slice(190, 240))
    gb = tos_nino34.groupby('time.month')
    tos_nino34_anom = gb - gb.mean(dim='time')
    index_nino34 = tos_nino34_anom.mean(dim=['lat', 'lon'])
    index_nino34_rolling_mean = index_nino34.rolling(time=5, center=True).mean()
    std_dev_tos = tos_nino34.std()
    normalized_index_nino34_rolling_mean = index_nino34_rolling_mean / std_dev_tos
    
    index_nino34_full.append(index_nino34)
    index_nino34_rolling_mean_full.append(index_nino34_rolling_mean)
    normalized_index_nino34_rolling_mean_full.append(normalized_index_nino34_rolling_mean)
    
    # Niño 4 (5N-5S, 170W-120W)
    tos_nino4 = SST.sel(lat=slice(-5, 5), lon=slice(160, 210))
    gb = tos_nino4.groupby('time.month')
    tos_nino4_anom = gb - gb.mean(dim='time')
    index_nino4 = tos_nino4_anom.mean(dim=['lat', 'lon'])
    std_dev_tos_4 = tos_nino4.std()
    normalized_index_nino4_mean = index_nino4 / std_dev_tos_4
    
    index_nino4_full.append(index_nino4)
    normalized_index_nino4_full.append(normalized_index_nino4_mean)
    
    # ONI (5N-5S, 170W-120W)
    tos_oni = SST.sel(lat=slice(-5, 5), lon=slice(190, 240))
    gb = tos_oni.groupby('time.month')
    tos_oni_anom = gb - gb.mean(dim='time')
    index_oni = tos_oni_anom.mean(dim=['lat', 'lon'])
    index_oni_rolling_mean = index_oni.rolling(time=3, center=True).mean()
    std_dev_oni = tos_oni.std()
    normalized_index_oni_rolling_mean = index_oni_rolling_mean / std_dev_oni
    
    index_oni_full.append(index_oni)
    index_oni_rolling_mean_full.append(index_oni_rolling_mean)
    normalized_index_oni_rolling_mean_full.append(normalized_index_oni_rolling_mean)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


...


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'



--> The keys in the returned dictionary of datasets are constructed as follows:
	'component.experiment.stream.forcing_variant.variable'


In [22]:
# nino12_first50 = xr.concat(index_nino12_full, "nino12")
# normalized_index_nino12_first50 = xr.concat(normalized_index_nino12_full, "normed_nino12")

# nino3_first50 = xr.concat(index_nino3_full, "nino3")
# normalized_index_nino3_first50 = xr.concat(normalized_index_nino3_full, "normed_nino3")

# nino34_first50 = xr.concat(index_nino34_full, "nino34")
# nino34_rolling_mean_first50 = xr.concat(index_nino34_rolling_mean_full, "rm_nino34")
# normalized_index_nino34_rolling_mean_first50 = xr.concat(normalized_index_nino34_rolling_mean_full, "normed_rm_nino34")

# nino4_first50 = xr.concat(index_nino4_full, "nino4")
# normalized_index_nino4_first50 = xr.concat(normalized_index_nino4_full, "normed_nino4")

# oni_first50 = xr.concat(index_oni_full, "oni")
# oni_rolling_mean_first50 = xr.concat(index_oni_rolling_mean_full, "rm_oni")
# normalized_index_oni_rolling_mean_first50 = xr.concat(normalized_index_oni_rolling_mean_full, "normed_rm_oni")

In [ ]:
nino12_last50 = xr.concat(index_nino12_full, "nino12")
normalized_index_nino12_last50 = xr.concat(normalized_index_nino12_full, "normed_nino12")

nino3_last50 = xr.concat(index_nino3_full, "nino3")
normalized_index_nino3_last50 = xr.concat(normalized_index_nino3_full, "normed_nino3")

nino34_last50 = xr.concat(index_nino34_full, "nino34")
nino34_rolling_mean_last50 = xr.concat(index_nino34_rolling_mean_full, "rm_nino34")
normalized_index_nino34_rolling_mean_last50 = xr.concat(normalized_index_nino34_rolling_mean_full, "normed_rm_nino34")

nino4_last50 = xr.concat(index_nino4_full, "nino4")
normalized_index_nino4_last50 = xr.concat(normalized_index_nino4_full, "normed_nino4")

oni_last50 = xr.concat(index_oni_full, "oni")
oni_rolling_mean_last50 = xr.concat(index_oni_rolling_mean_full, "rm_oni")
normalized_index_oni_rolling_mean_last50 = xr.concat(normalized_index_oni_rolling_mean_full, "normed_rm_oni")

In [25]:
# nino12_first50.to_netcdf(path='nino12_first50.nc')
# normalized_index_nino12_first50.to_netcdf(path='normalized_index_nino12_first50.nc')

# nino3_first50.to_netcdf(path='nino3_first50.nc')
# normalized_index_nino3_first50.to_netcdf(path='normalized_index_nino3_first50.nc')

# nino34_first50.to_netcdf(path='nino34_first50.nc')
# nino34_rolling_mean_first50.to_netcdf(path='nino34_rolling_mean_first50.nc')
# normalized_index_nino34_rolling_mean_first50.to_netcdf(path='normalized_index_nino34_rolling_mean_first50.nc')

# nino4_first50.to_netcdf(path='nino4_first50.nc')
# normalized_index_nino4_first50.to_netcdf(path='normalized_index_nino4_first50.nc')

# oni_first50.to_netcdf(path='oni_first50.nc')
# oni_rolling_mean_first50.to_netcdf(path='oni_rolling_mean_first50.nc')
# normalized_index_oni_rolling_mean_first50.to_netcdf(path='normalized_index_oni_rolling_mean_first50.nc')

In [ ]:
nino12_last50.to_netcdf(path='nino12_last50.nc')
normalized_index_nino12_last50.to_netcdf(path='normalized_index_nino12_last50.nc')

nino3_last50.to_netcdf(path='nino3_last50.nc')
normalized_index_nino3_last50.to_netcdf(path='normalized_index_nino3_last50.nc')

nino34_last50.to_netcdf(path='nino34_last50.nc')
nino34_rolling_mean_last50.to_netcdf(path='nino34_rolling_mean_last50.nc')
normalized_index_nino34_rolling_mean_last50.to_netcdf(path='normalized_index_nino34_rolling_mean_last50.nc')

nino4_last50.to_netcdf(path='nino4_last50.nc')
normalized_index_nino4_last50.to_netcdf(path='normalized_index_nino4_last50.nc')

oni_last50.to_netcdf(path='oni_last50.nc')
oni_rolling_mean_last50.to_netcdf(path='oni_rolling_mean_last50.nc')
normalized_index_oni_rolling_mean_last50.to_netcdf(path='normalized_index_oni_rolling_mean_last50.nc')

In [ ]:
# Niño 1+2 (5N-5S, 270W-90W)
tos_nino12 = SST.sel(lat=slice(0, 10), lon=slice(270, 280))
gb = tos_nino12.groupby('time.month')
tos_nino12_anom = gb - gb.mean(dim='time')
index_nino12 = tos_nino12_anom.mean(dim=['lat', 'lon'])
std_dev_tos_12 = tos_nino12.std()
normalized_index_nino12 = index_nino12 / std_dev_tos_12

In [ ]:
# Niño 3 (5N-5S, 150W-90W)
tos_nino3 = SST.sel(lat=slice(-5, 5), lon=slice(210, 270))
gb = tos_nino3.groupby('time.month')
tos_nino3_anom = gb - gb.mean(dim='time')
index_nino3 = tos_nino3_anom.mean(dim=['lat', 'lon'])
std_dev_tos_3 = tos_nino3.std()
normalized_index_nino3_mean = index_nino3 / std_dev_tos_3

In [ ]:
# Niño 4 (5N-5S, 170W-120W)
tos_nino4 = SST.sel(lat=slice(-5, 5), lon=slice(160, 210))
gb = tos_nino4.groupby('time.month')
tos_nino4_anom = gb - gb.mean(dim='time')
index_nino4 = tos_nino4_anom.mean(dim=['lat', 'lon'])
std_dev_tos_4 = tos_nino4.std()
normalized_index_nino4_mean = index_nino4 / std_dev_tos_4

In [ ]:
# Niño 3.4 (5N-5S, 170W-120W)
tos_nino34 = SST.sel(lat=slice(-5, 5), lon=slice(190, 240))
gb = tos_nino34.groupby('time.month')
tos_nino34_anom = gb - gb.mean(dim='time')
index_nino34 = tos_nino34_anom.mean(dim=['lat', 'lon'])
index_nino34_rolling_mean = index_nino34.rolling(time=5, center=True).mean()
std_dev_tos = tos_nino34.std()
normalized_index_nino34_rolling_mean = index_nino34_rolling_mean / std_dev_tos

In [ ]:
# ONI (5N-5S, 170W-120W)
tos_oni = SST.sel(lat=slice(-5, 5), lon=slice(190, 240))
gb = tos_oni.groupby('time.month')
tos_oni_anom = gb - gb.mean(dim='time')
index_oni = tos_oni_anom.mean(dim=['lat', 'lon'])
index_oni_rolling_mean = index_oni.rolling(time=3, center=True).mean()
std_dev_oni = tos_oni.std()
normalized_index_oni_rolling_mean = index_oni_rolling_mean / std_dev_oni

In [ ]:
index_nino34.plot(size=8)
index_nino34_rolling_mean.plot()
plt.legend(['anomaly', '5-month running mean anomaly'])
plt.title('SST anomaly over the Niño 3.4 region');

In [ ]:
fig = plt.figure(figsize=(12, 6))

plt.fill_between(
    normalized_index_nino34_rolling_mean.time.data,
    normalized_index_nino34_rolling_mean.where(
        normalized_index_nino34_rolling_mean >= 0.4
    ).data,
    0.4,
    color='red',
    alpha=0.9,
)
plt.fill_between(
    normalized_index_nino34_rolling_mean.time.data,
    normalized_index_nino34_rolling_mean.where(
        normalized_index_nino34_rolling_mean <= -0.4
    ).data,
    -0.4,
    color='blue',
    alpha=0.9,
)

normalized_index_nino34_rolling_mean.plot(color='black')
plt.axhline(0, color='black', lw=0.5)
plt.axhline(0.4, color='black', linewidth=0.5, linestyle='dotted')
plt.axhline(-0.4, color='black', linewidth=0.5, linestyle='dotted')
plt.title('Niño 3.4 Index');

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))
ax.coastlines()
ax.gridlines()
tos_nino34.isel(time=0).plot(ax=ax, transform=ccrs.PlateCarree(), vmin=-2, vmax=30, cmap='coolwarm')
ax.set_extent((120, 300, 10, -10))